In [2]:
import os
import urllib.request
import pandas as pd


#Download CSV data to get some data.
def dl_if_not_exists(url,filename):
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)
    
#WARNING: This is a lot of data. Skip to the third cell if you just want to download the pictures.
dl_if_not_exists("https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv", "train-images-boxable-with-rotation.csv")
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv",'class-descriptions-boxable.csv')
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/train-annotations-human-imagelabels-boxable.csv",'train-annotations-human-imagelabels-boxable.csv')
dl_if_not_exists("https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv",'oidv6-train-annotations-bbox.csv')

imagesId = pd.read_csv('train-images-boxable-with-rotation.csv',index_col=0)
imageLabels = pd.read_csv('train-annotations-human-imagelabels-boxable.csv',index_col=0)
boundingBoxes = pd.read_csv('oidv6-train-annotations-bbox.csv',index_col=0)
classDescriptions = pd.read_csv('class-descriptions-boxable.csv',header=None) \
                        .rename({0:'LabelName',1:'Class'},axis=1)

In [3]:
from functools import reduce
Urls = imagesId[['OriginalURL',"OriginalLandingURL"]]

#These are big cats image ids.
catsSet = ['/m/07dm6','/m/096mb','/m/0c29q','/m/0cd4d']

dfs = []
for c in catsSet:
    dfs.append(imageLabels[imageLabels['LabelName'] == c])
    
filteredLabels = reduce(lambda a,b : a.append(b),dfs)
filteredLabels = filteredLabels[filteredLabels['Confidence'] == 1][['LabelName']]

boundingBoxes = boundingBoxes[['LabelName','XMin','XMax','YMin','YMax']]

dfs = []
for c in catsSet:
    dfs.append(boundingBoxes[boundingBoxes['LabelName'] == c])
    
filteredBoxes = reduce(lambda a,b : a.append(b),dfs)

In [4]:
df1 = filteredLabels.join(Urls)
df2 = df1.merge(filteredBoxes,on=['ImageID','LabelName'])
df3 = df2.merge(classDescriptions,on=['LabelName'])
df3.pop('LabelName')
df3.pop('OriginalLandingURL')
df3.to_csv('big_cats_image_data.csv')

In [2]:
import pandas as pd
import urllib.request
import os
import asyncio
import shutil

files = pd.read_csv('big_cats_image_data.csv',header=[0])

#Download CSV data to get some data.
async def dl_if_not_exists_async(url):
    try:
        filename = "./bigCats/"+url[url.rfind('/')+1:]
        if not os.path.isfile(filename):
            urllib.request.urlretrieve(url, filename)
    except:
        pass
            

'''
for r in files.iloc:
    print(r.name,r.OriginalURL)
'''

if not os.path.isdir('bigCats'):
    os.mkdir('bigCats', 0o755)

tasks = [dl_if_not_exists_async(r.OriginalURL) for r in files.iloc]
await asyncio.wait(tasks)

({<Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-2-91975b47a059>:13> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-2-91975b47a059>:13> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-2-91975b47a059>:13> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-2-91975b47a059>:13> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-2-91975b47a059>:13> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-2-91975b47a059>:13> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-2-91975b47a059>:13> result=None>,
  <Task finis